In [2]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{SparkSession, DataFrame}

import org.apache.spark.sql.functions._
import org.apache.spark.sql.{SparkSession, DataFrame}


Запуск сессии

In [3]:
val spark = SparkSession.builder()
  .appName("Music")
  .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@44e45a5b


Загрузка датасета

In [6]:
val df = spark.read
  .format("csv")
  .option("header", "true")
  .load("final.csv") 

df.show()

+---+--------------------+----+---------------+-----------+-----------------+--------------------+-----------------+--------------------+------+--------------------+------------+----------------+--------------------+--------------------+---------+-------------+--------------+-------+----------+------------------+------------------+----+----+-------------------+-------------------+------------------+----------------+--------+-------------------+------------------+--------+---------+-------------+--------+-----+
|_c0|                 uri|rank|   artist_names|artists_num|artist_individual|           artist_id|     artist_genre|          artist_img|collab|          track_name|release_date|album_num_tracks|         album_cover|              source|peak_rank|previous_rank|weeks_on_chart|streams|      week|      danceability|            energy| key|mode|           loudness|        speechiness|      acousticness|instrumentalness|liveness|            valence|             tempo|duration|  countr

df: org.apache.spark.sql.DataFrame = [_c0: string, uri: string ... 34 more fields]


Top-10 artists by total streams.

In [8]:
df.select("artist_names", "track_name", "streams")
  .groupBy("artist_names")
  .agg(sum("streams").alias("total_streams"))
  .orderBy(desc("total_streams"))
  .limit(10)
  .show()

+--------------------+---------------+
|        artist_names|  total_streams|
+--------------------+---------------+
|          Ed Sheeran| 1.364386947E10|
|      Olivia Rodrigo|1.1741091136E10|
|           Bad Bunny|1.0379978355E10|
|       Billie Eilish|  9.870341236E9|
|          The Weeknd|  9.485489222E9|
|        Harry Styles|  9.124389514E9|
|       Ariana Grande|  8.190436653E9|
|               Drake|  7.812727326E9|
|The Kid LAROI, Ju...|   7.35161562E9|
|         Post Malone|  7.283596733E9|
+--------------------+---------------+



Top 5 songs with the highest number of weeks on the charts globally

In [10]:
df.filter(col("country") === "Global")
  .orderBy(desc("weeks_on_chart"))
  .select("track_name", "artist_names", "weeks_on_chart")
  .limit(5)
  .show()

+--------------------+-------------------+--------------+
|          track_name|       artist_names|weeks_on_chart|
+--------------------+-------------------+--------------+
|          High Hopes|Panic! At The Disco|            99|
|when the party's ...|      Billie Eilish|            99|
|      Mr. Brightside|        The Killers|            99|
|             Heather|         Conan Gray|            99|
|          goosebumps|       Travis Scott|            99|
+--------------------+-------------------+--------------+



The average danceability, energy, and valence of songs released in 2022

In [11]:
df.filter(year(col("release_date")) === 2022)
  .agg(avg("danceability").alias("avg_danceability"),
       avg("energy").alias("avg_energy"),
       avg("valence").alias("avg_valence"))
       .show()

+------------------+------------------+------------------+
|  avg_danceability|        avg_energy|       avg_valence|
+------------------+------------------+------------------+
|0.7109250632167036|0.6633135773008008|0.5631852331667105|
+------------------+------------------+------------------+



Top 10 artists with the highest average streams per week

In [12]:
df.groupBy("artist_names")
  .agg(avg("streams").alias("avg_streams"))
  .orderBy(desc("avg_streams"))
  .limit(10)
  .show()

+--------------------+--------------------+
|        artist_names|         avg_streams|
+--------------------+--------------------+
|StaySolidRocky, L...|        1.75541045E7|
|Luis Fonsi, Daddy...|   1.7091451234375E7|
|Anuel AA, Daddy Y...|1.4687811955555556E7|
|Taylor Swift, The...|         1.4385703E7|
| Nicky Jam, J Balvin|1.4219319205882354E7|
|DJ Khaled, Justin...|1.4134057901960785E7|
|Lil Nas X, Billy ...| 1.366073164864865E7|
|DJ Khaled, Justin...| 1.362010794736842E7|
|Drake, Michael Ja...|     1.34909444375E7|
|6ix9ine, Nicki Mi...|        1.34590289E7|
+--------------------+--------------------+



The most common genres among artists in the dataset

In [21]:
df.groupBy("artist_genre")
  .count()
  .orderBy(desc("count"))
  .show()

+--------------------+------+
|        artist_genre| count|
+--------------------+------+
|                 pop|156910|
|                   0|114293|
|         trap latino| 86138|
|           reggaeton| 73498|
|               latin| 73176|
|           dance pop| 49390|
|                 rap| 31621|
|              uk pop| 27131|
|        canadian pop| 20416|
|               k-pop| 19415|
|reggaeton colombiano| 19321|
|           latin pop| 16922|
|      reggaeton flow| 16003|
|             hip hop| 15572|
|       latin hip hop| 14813|
|             pop rap| 14719|
|                trap| 14044|
|      trap argentino| 13080|
|      german hip hop| 12529|
|           pop dance| 12021|
+--------------------+------+
only showing top 20 rows



The average number of streams per week for each country

In [22]:
df.groupBy("country")
  .agg(avg("streams").alias("avg_streams_per_week"))
  .orderBy(desc("avg_streams_per_week"))
  .show()

+--------------+--------------------+
|       country|avg_streams_per_week|
+--------------+--------------------+
|        Global|    8590443.78178008|
| United States|  2612468.3299010973|
|        Brazil|  1541791.7800724516|
|        Mexico|  1317617.0092200846|
|       Germany|    784578.343812709|
|         Spain|    713261.081671159|
|         India|   709459.8710540024|
|United Kingdom|   696611.6486762615|
|         Italy|   624623.5501046622|
|        France|   584984.9326468467|
|     Argentina|   547315.6179943004|
|     Indonesia|   542640.4211397746|
|        Turkey|  502700.14107988315|
|         Chile|   456507.3099201683|
|   Philippines|  455304.44186168537|
|         Japan|   437043.5067144136|
|     Australia|  394437.60274980456|
|        Canada|   385216.4609696333|
|   Netherlands|   369540.7430649392|
|        Poland|   356572.3758571201|
+--------------+--------------------+
only showing top 20 rows



Total number of unique artists in each region

In [24]:
df.groupBy("region")
  .agg(countDistinct("artist_names").alias("unique_artists"))
  .orderBy(desc("unique_artists"))
  .show()

+---------------+--------------+
|         region|unique_artists|
+---------------+--------------+
|         Europe|         11085|
|           Asia|          3587|
|         Global|          2500|
|  South America|          2051|
|    Middle East|          2030|
|         Africa|          1796|
|  North America|          1502|
|Central America|           975|
|        Oceania|           812|
|      Caribbean|           663|
|        Ukraine|             2|
|       Paraguay|             1|
|           0.11|             1|
|         Taiwan|             1|
|         207394|             1|
|  United States|             1|
|         region|             1|
|         Mexico|             1|
|      Guatemala|             1|
|       Honduras|             1|
+---------------+--------------+
only showing top 20 rows



The average duration of songs in each language

In [31]:
df.groupBy("language")
  .agg(avg("duration").alias("avg_duration"))
  .orderBy(desc("avg_duration"))
  .show()

+----------+------------------+
|  language|      avg_duration|
+----------+------------------+
|      Zulu| 277846.9191161834|
|  Japanese|236926.38478066248|
|  Mandarin|229092.00780014182|
|   Spanish| 229090.1674511652|
| Cantonese| 225446.2013226366|
|Indonesian|223818.99283614007|
|   Tagalog|223522.45890338876|
|Vietnamese|222888.56846780164|
|     Hindi|222448.41670428895|
|Portuguese| 220139.6228857533|
|      Urdu|219468.59743918054|
|      Thai|216450.96938240537|
|    Global|211137.86207397297|
|    Korean|209461.11974094732|
| Icelandic|205727.16012183693|
|   English|205184.59067171466|
|     Malay|204851.87832109997|
|    Arabic|204682.33773191096|
|    French|203644.29232546827|
|    Hebrew|202510.22794735368|
+----------+------------------+
only showing top 20 rows



The most common source (record label) of songs in the dataset

In [32]:
df.groupBy("source")
  .count()
  .orderBy(desc("count"))
  .show()

+--------------------+-----+
|              source|count|
+--------------------+-----+
|            Columbia|79751|
|Rimas Entertainme...|59518|
|    Sony Music Latin|49662|
|    Republic Records|40146|
| Atlantic Records UK|32368|
|    Atlantic Records|27894|
|       UMLE - Latino|24925|
|          WEA Latina|23544|
|      Warner Records|23409|
|Sony Music Entert...|22119|
|Kemosabe Records/...|21062|
|   RCA Records Label|20477|
|          Rich Music|18672|
|        RBMG/Def Jam|18278|
|   Olivia Rodrigo PS|16626|
|                 OVO|12814|
|Sony Music Latin/...|12369|
|     Polydor Records|12184|
|Ministry of Sound...|11942|
|                 EMI|11259|
+--------------------+-----+
only showing top 20 rows



Top 10 artists with the highest number of collaborations

In [38]:
df.filter(col("collab") === 1)
  .groupBy("artist_names")
  .agg(countDistinct("track_name").alias("collaborations"))
  .orderBy(desc("collaborations"))
  .limit(10)
  .show()

+--------------------+--------------+
|        artist_names|collaborations|
+--------------------+--------------+
|          Ego, Coder|            20|
|       Bedoes, Lanek|            15|
|Marília Mendonça,...|            15|
|Big Baby Tape, ki...|            14|
|Kaaris, Kalash Cr...|            14|
|Benny Jamz, Gilli...|            12|
|      Azahriah, DESH|            12|
|     Anuel AA, Ozuna|            12|
|AKC Misi, AKC Kretta|            11|
|       Stormy, Tagne|            11|
+--------------------+--------------+



In [39]:
spark.stop()